In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
import platform

# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

## 트리의 앙상블

In [2]:
# 정형데이터에 가장 뛰어난 성능을 보이는 모델
# 앙상블 모델들은 결정트리(Decision Tree)를 기반으로 만들어 졌습니다
# 앙상블 모델 종류
    # - 랜덤포레스트(Random Forest)
    # - 엑스트라 트리 (Extra Trees)
    # - 그레디언트 부스팅(Gradient Boosting)
    # - 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Boosting)

## 랜덤 포레스트(Random Forest)
- 앙상블 모델 중에 가장 대표격 모델
    - 안정적인 성능
    - 앙상블 모델중 가장 먼저 시도
    - 훈련데이터 과대적합 방지
    - 검증데이터, 테스트데이터 안정적인 성능
    <br><br>

- 학습개념
    - 결정 트리를 랜덤하게 만들어서 숲을 만드는 느낌
    - 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련을 완료한 후
    - 다시 원본 훈련데이터에 반환
    - 랜덤하게 추출 시 이전에 사용된 샘플을 사용 할 수도 있다(중복 허용)
    <br><br>
    
- 부트스트랩 샘플(Bootstrap Sample)
    - 위에 설명한 랜덤한 샘플 추출 시 중복 허용 > 데이터 샘플링
    - 샘플 추출 방식
        - 1. 원본에서 랜덤 샘플 추출
        - 2. 훈련 이후 사용이 끝나면 원본에 반환
        - 3. 다시 원본에서 샘플 추출, 이때 중복값 추출 될 수 있음
        - 4. 위 순서를 반복하면서 샘플링을 통해 훈련하는 방식

In [3]:
from sklearn.model_selection import train_test_split

wine = pd.read_csv('./data/08_wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
train_input, test_input, train_target, test_target = \
    train_test_split(data, target, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(4872, 3) (4872,)
(1625, 3) (1625,)


## 훈련모델 생성하기

In [6]:
# 랜덤포레스트 패키지 : sklearn.ensemble
# 랜덤포레스트 클래스(모델) : RandomForestClassifir
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

In [7]:
# 교차검증 : crocross_validate()
# 교차검증 후 훈련 검증결과와 테스트 검증결과 확인하기
from sklearn.model_selection import cross_validate

# return_train_score=True : 검증결과 반환
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print('<랜덤포레스트>   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<랜덤포레스트> 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<랜덤포레스트>   훈련데이터 정확도 : 0.9978
<랜덤포레스트> 테스트데이터 정확도 : 0.8914


In [8]:
# 특성 중요도 조회하기
rf.fit(train_input, train_target)
print('[alcohol, sugar, pH]')
print(rf.feature_importances_)

[alcohol, sugar, pH]
[0.23155241 0.49706658 0.27138101]


In [9]:
# oob 기능 사용
# 훈련에 참여하지 못한 잔여 샘플 사용하는 기능
# 기본은 사용 x
rf = RandomForestClassifier(oob_score= True, n_jobs= -1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8981937602627258


## 엑스트라 트리(Extra Tree)
- 랜덤 포레스트와 유사하게 작동
- 기본적으로 100개의 결정트리를 훈련함
- 랜덤포레스트와의 차이점
    - 부트스트랩 샘플링을 지원하지 않음
    - 훈련데이터 전체를 이용하여 결정트리를 생성
    - 무작위로 트리를 분리함
- 사용되는 속성 : splitter = 'random' 무작위속성
- 장점
    - 과대적합을 막고, 검증데이터의 평가 값을 높일 수 있음
    - 특성 데이터가 많지 않은 경우에는 랜덤 포레스트와 큰 차이가 없음
- 랜덤포레스트는 불순도 등 여러가지 조건에 따라 결정트리를 생성하기 때문에 속도가 느린 반면
- 엑스트라 트리는 랜덤하게 결정트리를 생성하기에 속도가 다소 빠르다는 장점이 있다

In [10]:
# 사용 패키지 : 랜덤포레스트와 동일
# 사용되는 클래스(모델) : ExtraTreeClassifier
# 코어 전체 사용, train 및 test 결과값 출력
# 교차 검증 결과인 train 및 test 결과 확인
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(et, train_input, train_target, 
                        return_train_score= True,n_jobs=-1)

print('<엑스트라트리>   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<엑스트라트리> 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<엑스트라트리>   훈련데이터 정확도 : 0.9978
<엑스트라트리> 테스트데이터 정확도 : 0.8904


In [11]:
# 특성 중요도 조회하기
et.fit(train_input, train_target)
print('[alcohol, sugar, pH]')
print(et.feature_importances_)

[alcohol, sugar, pH]
[0.20702369 0.51313261 0.2798437 ]


## 그레디언트 부스팅(Gradient Boosting)
- 기존 훈련모델의 오차를 많이 보완
- 성능 향상을 위한 모델
- 과대적합, 일반화(과적합이 없는 상태)에 강함
- 깊이(max_depth)가 앝은 결정트리를 사용함
     - 기본적으로 max_depth = 3 사용
     - 결정트리는 100개 사용
<br><br>
- 성능향상 테스트 방법
    - 결정트리의 갯수를 조절하면서 테스트
    - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
    - 기본 학습률은 0.1
<br><br>
- 단점
    - 순서대로 트리를 추가(랜덤하지 않음)하지 않기 때문에 훈련속도 느림
    - 이런속도를 개선한 모델 > '히스토그램 기반 그레디언트 부스팅' 모델

In [12]:
# 그레디언트 부스팅 모델 생성
# 사용하는 클래스(모델 : GradientBoostingClassifier
    # 객체생성시 seed값만 부여
    # 교차 검증시 train, test 결과값 출력
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)

scores = cross_validate(gb, train_input, train_target,
                        return_train_score= True, n_jobs=-1)

print('<그레디언트>   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<그레디언트> 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<그레디언트>   훈련데이터 정확도 : 0.8895
<그레디언트> 테스트데이터 정확도 : 0.8715


In [13]:
gb.fit(train_input, train_target)
print('[alcohol, sugar, pH]')
print(gb.feature_importances_)

[alcohol, sugar, pH]
[0.12517641 0.73300095 0.14182264]


In [14]:
gb.score(test_input, test_target)

0.8578461538461538

## 학습률 적용하기

In [15]:
# 학습률이 커지면 트리 보정을 강하게 하기 때문에
# 복잡한 모델을 만들어서 일반화 성능을 떨어뜨리게 된다
# 학습률 : learning_rate = 0.1(기본값)
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators= 100,
                                learning_rate=0.1,
                                random_state=42)

scores = cross_validate(gb, train_input, train_target,
                        return_train_score= True, n_jobs=-1)

print('<그레디언트>   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<그레디언트> 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<그레디언트>   훈련데이터 정확도 : 0.8895
<그레디언트> 테스트데이터 정확도 : 0.8715


## 히스토그램 기반 그레디언트 부스팅(HistGradientBoostingClassifier) 
- 사용하는 클래스(모델) : HistGradientBoostingClassifier

In [16]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target,
                        return_train_score= True, n_jobs=-1)

print('<히스토그램 그레디언트>   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<히스토그램 그레디언트> 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<히스토그램 그레디언트>   훈련데이터 정확도 : 0.938
<히스토그램 그레디언트> 테스트데이터 정확도 : 0.8805


In [17]:
hgb.fit(train_input, train_target)
hgb.score(test_input, test_target)

0.8584615384615385

## sklearn 이외의 히스토그램 기반 그레디언트 부스팅

## XGBoost

In [18]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method ='hist', random_state=42)

scores = cross_validate(xgb, train_input, train_target,
                        return_train_score= True, n_jobs=-1)

print('<XGBoost>   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<XGBoost> 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<XGBoost>   훈련데이터 정확도 : 0.9614
<XGBoost> 테스트데이터 정확도 : 0.8834


## LightGBM

In [20]:
# 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 패키지
# - 훈련속도가 매우 빠름
# - 최신 기술을 많이 적용하고 있어 인기가 증가하는 추세
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)

scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print('<LightGBM>   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<LightGBM> 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<LightGBM>   훈련데이터 정확도 : 0.9413
<LightGBM> 테스트데이터 정확도 : 0.8846
